# Setting up and running a simulation in which synthesis is perturbed

First we must import both a `LinearModel` and a `ConditionSimulation` object from the `promoters` package.

In [ ]:
from promoters.models.linear import LinearModel
from promoters.simulation.environment import ConditionSimulation

Next we need to define our model along with the activation strengths for each promoter. These are equivalent to $\eta_1$, $\eta_2$, and $\eta_3$ as defined in the manuscript. For our example model, we will assume a cell contains a single redundant promoter acting upon the target protein.

In [ ]:
# perturbation severity
severity = 0.5

# promoter strengths
eta = (0,0,1*severity)

# define base model
model = LinearModel(g1=0.01, g2=0.001, include_activation=True)

# add promoters subject to perturbation
model.add_promoters(*eta, perturbed=True)

We then use the `ConditionSimulation.run` method to run the simulation under each metabolic condition.

In [ ]:
# run simulation
simulation = ConditionSimulation(model, )
simulation.run(skwargs=dict(N=100))

And then we can use `ConditionSimulation.plot_comparison` method to visualize the result.

In [ ]:
# plot result
simulation.plot_comparison()

# Visualizing Individual Simulation Trajectories

Upon completing a simulation, instances of `ConditionSimulation` acquire a `comparisons` attribute. This is a dictionary of `Comparison` objects keyed by metabolic condition. Each comparison object contains `reference` and `compared` attributes, which are `genessa.TimeSeries` instances containing the simulated dynamics before and after the promoters are removed. Error frequencies are accessible via the `Comparison.threshold_error` attribute.

The function below will add a randomly sampled subset of trajectories simulated before and after the promoters are removed to a predefined axis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_trajectories(comparison, ax, N=10):
    """ Plot <N> trajectories from each condition in <comparison> on <ax>. """
    
    # plot <N> randomly selected trajectories before and after all repression is removed
    ind = np.random.randint(0, comparison.reference.states.shape[0], N)
    for trajectory in comparison.reference.states[ind, -1, :]:
        ax.plot(comparison.t, trajectory, color='m', lw=0.5)
    for trajectory in comparison.compared.states[ind, -1, :]:
        ax.plot(comparison.t, trajectory, color='grey', lw=0.5)
    
    # format axis
    ax.set_ylabel('Protein level')
    ax.set_xlabel('Time (h)')
    ax.set_xlim(-2, 50)

We can then use the above method to visualize a given series of simulated trajectories.

In [ ]:
# get comparison simulated under normal metabolic conditions
comparison = simulation.comparisons['normal']

# plot simulated dynamics
fig, ax = plt.subplots(figsize=(2., 1.25))
plot_trajectories(comparison, ax)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_title('Normal metabolism')

# report error frequency
print('Error frequency: {:2.2%}'.format(comparison.threshold_error))
normal_ylim = ax.get_ylim()

In [ ]:
# get comparison simulated under normal metabolic conditions
comparison = simulation.comparisons['diabetic']

# plot simulated dynamics
fig, ax = plt.subplots(figsize=(2., 1.25))
plot_trajectories(comparison, ax)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_title('Slow metabolism')

# report error frequency
print('Error frequency: {:2.2%}'.format(comparison.threshold_error))
normal_ylim = ax.get_ylim()

In [ ]:
# get comparison simulated under normal metabolic conditions
comparison = simulation.comparisons['hyper_metabolic']

# plot simulated dynamics
fig, ax = plt.subplots(figsize=(2., 1.25))
plot_trajectories(comparison, ax)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_title('Fast metabolism')

# report error frequency
print('Error frequency: {:2.2%}'.format(comparison.threshold_error))
normal_ylim = ax.get_ylim()